<h2><center><font color="black"><strong>OLS</strong></font></center></h2>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [4]:
import statsmodels.api as sm
import math
from sklearn.metrics import mean_squared_error

In [79]:
df = pd.read_csv('../Data/update_data/df_Paris_Processed_Michel.csv', low_memory = False, encoding = 'latin1')


In [80]:
%run '../Helpers/preprocessing.py'
%run '../Helpers/graph.py'
%run '../Helpers/score.py'

In [81]:
    # Créer les variables en log
log_var(df, ['Prix_m2'])
log_var(df, ['valeur_fonciere'])
log_var(df, ['surface_reelle_bati'])
    # Ajouter la variable ANNEE-TRIMESTRE
annee_trimestre(df)

## Preprocessing

Ajouter des dummies pour les variables continues.  
Si 0 -> 0  
Si au moins 1 -> 1

In [82]:
def dummies_0_1(data, var) : 
    var_dummy = var + '_dummy'
    data[var_dummy] = 0
    data.loc[data[var] > 0, var_dummy] = 1

In [83]:
dummies_0_1(df, 'Nombre_dependance')

Ajouter des dummies pour les variables catégorielles.

In [84]:
df.Nombre_dependance_dummy.value_counts()

0    103693
1     40689
Name: Nombre_dependance_dummy, dtype: int64

In [85]:
df.iris_code = df.iris_code.astype(str)
df.num_trimestre = df.num_trimestre.astype(str)

features_OLS = ['nombre_pieces_principales', 'Nombre_dependance', 'Vente_par_iris_tri',
               'IPL_{Q-1}', 'N_proprietaire', 'N_vacant', 'N_apparts',
               'n_metros_iris']


X = df[['iris_code', 'num_trimestre'] + features_OLS + ['Prix_m2', 'date_mutation']]
df = pd.get_dummies(data=X, drop_first=True)

In [68]:
####  ATTENTION :  Gérer les catégories de référence

In [86]:
df.head(4)

,nombre_pieces_principales,Nombre_dependance,Vente_par_iris_tri,IPL_{Q-1},N_proprietaire,N_vacant,N_apparts,n_metros_iris,Prix_m2,date_mutation,iris_code_751010102,iris_code_751010103,iris_code_751010201,iris_code_751010202,iris_code_751010203,iris_code_751010204,iris_code_751010206,iris_code_751010301,iris_code_751010302,iris_code_751010303,iris_code_751010401,iris_code_751010402,iris_code_751020501,iris_code_751020502,iris_code_751020503,iris_code_751020601,iris_code_751020602,iris_code_751020701,iris_code_751020702,iris_code_751020703,iris_code_751020704,iris_code_751020801,iris_code_751020802,iris_code_751020803,iris_code_751020804,iris_code_751020805,iris_code_751030901,iris_code_751030902,iris_code_751030903,iris_code_751030904,iris_code_751030905,iris_code_751031001,iris_code_751031002,iris_code_751031003,iris_code_751031004,iris_code_751031101,iris_code_751031102,iris_code_751031103,iris_code_751031104,iris_code_751031201,iris_code_751031202,iris_code_751031203,iris_code_751031204,iris_code_751041301,iris_code_751041302,iris_code_751041303,iris_code_751041304,iris_code_751041401,iris_code_751041402,iris_code_751041403,iris_code_751041404,iris_code_751041501,iris_code_751041502,iris_code_751041503,iris_code_751041504,iris_code_751041601,iris_code_751041602,iris_code_751051701,iris_code_751051702,iris_code_751051703,iris_code_751051704,iris_code_751051705,iris_code_751051801,iris_code_751051802,iris_code_751051803,iris_code_751051804,iris_code_751051805,iris_code_751051806,iris_code_751051807,iris_code_751051808,iris_code_751051901,iris_code_751051902,iris_code_751051903,iris_code_751051904,iris_code_751051905,iris_code_751051906,iris_code_751051907,iris_code_751051908,iris_code_751052001,iris_code_751052002,iris_code_751052003,iris_code_751052004,iris_code_751052005,iris_code_751062101,iris_code_751062102,iris_code_751062103,iris_code_751062202,iris_code_751062203,iris_code_751062204,iris_code_751062301,iris_code_751062302,iris_code_751062303,iris_code_751062304,iris_code_751062305,iris_code_751062306,iris_code_751062307,iris_code_751062308,iris_code_751062309,iris_code_751062310,iris_code_751062311,iris_code_751062401,iris_code_751062402,iris_code_751072501,iris_code_751072502,iris_code_751072503,iris_code_751072504,iris_code_751072505,iris_code_751072506,iris_code_751072601,iris_code_751072602,iris_code_751072603,iris_code_751072604,iris_code_751072701,iris_code_751072702,iris_code_751072703,iris_code_751072704,iris_code_751072705,iris_code_751072706,iris_code_751072801,iris_code_751072802,iris_code_751072803,iris_code_751072804,iris_code_751072805,iris_code_751072806,iris_code_751072807,iris_code_751072808,iris_code_751072809,iris_code_751072810,iris_code_751072811,iris_code_751082901,iris_code_751082902,iris_code_751082903,iris_code_751082904,iris_code_751082905,iris_code_751083001,iris_code_751083002,iris_code_751083003,iris_code_751083004,iris_code_751083005,iris_code_751083006,iris_code_751083101,iris_code_751083102,iris_code_751083103,iris_code_751083104,iris_code_751083105,iris_code_751083106,iris_code_751083107,iris_code_751083108,iris_code_751083201,iris_code_751083202,iris_code_751083203,iris_code_751083204,iris_code_751083205,iris_code_751083206,iris_code_751083207,iris_code_751083208,iris_code_751083209,iris_code_751093301,iris_code_751093302,iris_code_751093303,iris_code_751093304,iris_code_751093305,iris_code_751093306,iris_code_751093307,iris_code_751093308,iris_code_751093309,iris_code_751093310,iris_code_751093311,iris_code_751093401,iris_code_751093403,iris_code_751093405,iris_code_751093407,iris_code_751093408,iris_code_751093409,iris_code_751093501,iris_code_751093502,iris_code_751093503,iris_code_751093504,iris_code_751093601,iris_code_751093602,iris_code_751093603,iris_code_751093604,iris_code_751093605,iris_code_751093606,iris_code_751093607,iris_code_751093608,iris_code_751093609,iris_code_751093610,iris_code_751103701,iris_code_751103702,iris_code_751103703,iris_code_751103704,iris_code_751103

In [106]:
# Définition du dataset
# Sélectionner Paris sur les appartements (2) + sélection sur les quantiles
df_model = filter_quantile(df, 'Prix_m2', 0.1, 0.99)



    # Définition des var explicatives
features_OLS = list(df_model.columns)
    # Def de l'output
label = 'Prix_m2'

In [107]:
df_model.head(3)

,nombre_pieces_principales,Nombre_dependance,Vente_par_iris_tri,IPL_{Q-1},N_proprietaire,N_vacant,N_apparts,n_metros_iris,Prix_m2,date_mutation,iris_code_751010102,iris_code_751010103,iris_code_751010201,iris_code_751010202,iris_code_751010203,iris_code_751010204,iris_code_751010206,iris_code_751010301,iris_code_751010302,iris_code_751010303,iris_code_751010401,iris_code_751010402,iris_code_751020501,iris_code_751020502,iris_code_751020503,iris_code_751020601,iris_code_751020602,iris_code_751020701,iris_code_751020702,iris_code_751020703,iris_code_751020704,iris_code_751020801,iris_code_751020802,iris_code_751020803,iris_code_751020804,iris_code_751020805,iris_code_751030901,iris_code_751030902,iris_code_751030903,iris_code_751030904,iris_code_751030905,iris_code_751031001,iris_code_751031002,iris_code_751031003,iris_code_751031004,iris_code_751031101,iris_code_751031102,iris_code_751031103,iris_code_751031104,iris_code_751031201,iris_code_751031202,iris_code_751031203,iris_code_751031204,iris_code_751041301,iris_code_751041302,iris_code_751041303,iris_code_751041304,iris_code_751041401,iris_code_751041402,iris_code_751041403,iris_code_751041404,iris_code_751041501,iris_code_751041502,iris_code_751041503,iris_code_751041504,iris_code_751041601,iris_code_751041602,iris_code_751051701,iris_code_751051702,iris_code_751051703,iris_code_751051704,iris_code_751051705,iris_code_751051801,iris_code_751051802,iris_code_751051803,iris_code_751051804,iris_code_751051805,iris_code_751051806,iris_code_751051807,iris_code_751051808,iris_code_751051901,iris_code_751051902,iris_code_751051903,iris_code_751051904,iris_code_751051905,iris_code_751051906,iris_code_751051907,iris_code_751051908,iris_code_751052001,iris_code_751052002,iris_code_751052003,iris_code_751052004,iris_code_751052005,iris_code_751062101,iris_code_751062102,iris_code_751062103,iris_code_751062202,iris_code_751062203,iris_code_751062204,iris_code_751062301,iris_code_751062302,iris_code_751062303,iris_code_751062304,iris_code_751062305,iris_code_751062306,iris_code_751062307,iris_code_751062308,iris_code_751062309,iris_code_751062310,iris_code_751062311,iris_code_751062401,iris_code_751062402,iris_code_751072501,iris_code_751072502,iris_code_751072503,iris_code_751072504,iris_code_751072505,iris_code_751072506,iris_code_751072601,iris_code_751072602,iris_code_751072603,iris_code_751072604,iris_code_751072701,iris_code_751072702,iris_code_751072703,iris_code_751072704,iris_code_751072705,iris_code_751072706,iris_code_751072801,iris_code_751072802,iris_code_751072803,iris_code_751072804,iris_code_751072805,iris_code_751072806,iris_code_751072807,iris_code_751072808,iris_code_751072809,iris_code_751072810,iris_code_751072811,iris_code_751082901,iris_code_751082902,iris_code_751082903,iris_code_751082904,iris_code_751082905,iris_code_751083001,iris_code_751083002,iris_code_751083003,iris_code_751083004,iris_code_751083005,iris_code_751083006,iris_code_751083101,iris_code_751083102,iris_code_751083103,iris_code_751083104,iris_code_751083105,iris_code_751083106,iris_code_751083107,iris_code_751083108,iris_code_751083201,iris_code_751083202,iris_code_751083203,iris_code_751083204,iris_code_751083205,iris_code_751083206,iris_code_751083207,iris_code_751083208,iris_code_751083209,iris_code_751093301,iris_code_751093302,iris_code_751093303,iris_code_751093304,iris_code_751093305,iris_code_751093306,iris_code_751093307,iris_code_751093308,iris_code_751093309,iris_code_751093310,iris_code_751093311,iris_code_751093401,iris_code_751093403,iris_code_751093405,iris_code_751093407,iris_code_751093408,iris_code_751093409,iris_code_751093501,iris_code_751093502,iris_code_751093503,iris_code_751093504,iris_code_751093601,iris_code_751093602,iris_code_751093603,iris_code_751093604,iris_code_751093605,iris_code_751093606,iris_code_751093607,iris_code_751093608,iris_code_751093609,iris_code_751093610,iris_code_751103701,iris_code_751103702,iris_code_751103703,iris_code_751103704,iris_code_751103

In [109]:
features_OLS = df_model.drop(['Nombre_dependance', 'Prix_m2', 'date_mutation'], axis = 1).columns

In [110]:
# Split des données en TRAIN et TEST
X_train, X_test, y_train, y_test = split_temporel_V1(df_model, features_OLS, label)

In [111]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [112]:
model = sm.OLS(y_train, X_train).fit()

In [113]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                Prix_m2   R-squared:                       0.440
Model:                            OLS   Adj. R-squared:                  0.435
Method:                 Least Squares   F-statistic:                     90.92
Date:                Tue, 14 Mar 2023   Prob (F-statistic):               0.00
Time:                        19:03:08   Log-Likelihood:            -8.9954e+05
No. Observations:              102772   AIC:                         1.801e+06
Df Residuals:                  101890   BIC:                         1.809e+06
Df Model:                         881                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 